In [1]:
import pandas as pd
import pickle

In [2]:
df = pd.read_csv('phil_nlp.csv')

df.sample(5)

,title,author,school,sentence_spacy,sentence_str,original_publication_date,corpus_edition_date,sentence_length,sentence_lowered,tokenized_txt,lemmatized_str
52043,Aristotle - Complete Works,Aristotle,aristotle,This then is the nature of concoction: inconco...,This then is the nature of concoction: inconco...,-320,1991,119,this then is the nature of concoction: inconco...,"['this', 'then', 'is', 'the', 'nature', 'of', ...",this then be the nature of concoction : incon...
168341,The Logic Of Scientific Discovery,Popper,analytic,The conditions under which this theorem holds ...,The conditions under which this theorem holds ...,1959,2002,133,the conditions under which this theorem holds ...,"['the', 'conditions', 'under', 'which', 'this'...",the condition under which this theorem hold c...
121617,The Search After Truth,Malebranche,rationalism,"Men imagine, then, that things of a different ...","Men imagine, then, that things of a different ...",1674,1997,148,"men imagine, then, that things of a different ...","['men', 'imagine', 'then', 'that', 'things', '...","man imagine , then , that thing of a differen..."
228846,The Crisis Of The European Sciences And Phenom...,Husserl,phenomenology,"Yet this is not, like memory in the usual sens...","Yet this is not, like memory in the usual sens...",1936,1970,161,"yet this is not, like memory in the usual sens...","['yet', 'this', 'is', 'not', 'like', 'memory',...","yet this be not , like memory in the usual se..."
73380,Aristotle - Complete Works,Aristotle,aristotle,(For things which are in the same genus must b...,(For things which are in the same genus must b...,-320,1991,139,(for things which are in the same genus must b...,"['for', 'things', 'which', 'are', 'in', 'the',...",( for thing which be in the same genus must b...


In [3]:
len(df)

369590

In [4]:
for_db = df
for_db['date'] = for_db['original_publication_date']
for_db['date'] = for_db['date'].apply(lambda x: str(x)[1:]+' BC' if x < 0 else str(x))
for_db['sentence'] = for_db['sentence_str']
for_db['school'] = for_db['school'].apply(lambda x: x.replace('_', ' ').title())
for_db = for_db.drop(['sentence_spacy', 
                      'sentence_length',
                      'sentence_lowered', 
                      'sentence_str', 
                      'tokenized_txt', 
                      'lemmatized_str',
                      'corpus_edition_date',
                      'original_publication_date'], axis=1)
for_db.columns = [i.upper() for i in for_db.columns]

for_db.sample(5)


,TITLE,AUTHOR,SCHOOL,DATE,SENTENCE
351250,Proslogion,Anselm,Scholasticism,1077,"Marco Polo asserted that for the Tibetans, 'no..."
119864,The Search After Truth,Malebranche,Rationalism,1674,The one easily notes the differences among thi...
240550,Being And Time,Heidegger,Phenomenology,1927,and earlier when. . .
146982,Lewis - Papers,Lewis,Analytic,1985,and ') correspond to
102497,A Treatise Of Human Nature,Hume,Empiricism,1739,"We know, that were we to approach equally near..."


In [5]:
for_db.sample(5)

,TITLE,AUTHOR,SCHOOL,DATE,SENTENCE
41009,Aristotle - Complete Works,Aristotle,Aristotle,320 BC,"A is true of, similarly with believing."
349256,Proslogion,Anselm,Scholasticism,1077,"Besides, because there is no original descript..."
135091,The Problems Of Philosophy,Russell,Analytic,1912,Hence also the free intellect will value more ...
190102,History Of Madness,Foucault,Continental,1961,One of these poor unfortunates was sleeping on...
259508,Critique Of Pure Reason,Kant,German Idealism,1781,But to say that they exist prior to all my exp...


In [6]:
for_db['SCHOOL'].value_counts(normalize=True)

Analytic           0.149963
Aristotle          0.131981
German Idealism    0.113997
Plato              0.103807
Continental        0.091396
Phenomenology      0.077310
Rationalism        0.062093
Empiricism         0.053927
Scholasticism      0.050423
Capitalism         0.049228
Communism          0.048589
Feminism           0.028410
Stoicism           0.020439
Nietzsche          0.018437
Name: SCHOOL, dtype: float64

In [7]:
classifier_dict = {}
for author in for_db['AUTHOR'].unique():
  classifier_dict[author] = 'AUTHOR'
for title in for_db['TITLE'].unique():
  classifier_dict[title] = 'TITLE'
for school in for_db['SCHOOL'].unique():
  classifier_dict[school] = 'SCHOOL'

In [8]:
classifier_dict

{'Plato': 'SCHOOL',
 'Aristotle': 'SCHOOL',
 'Locke': 'AUTHOR',
 'Hume': 'AUTHOR',
 'Berkeley': 'AUTHOR',
 'Spinoza': 'AUTHOR',
 'Leibniz': 'AUTHOR',
 'Descartes': 'AUTHOR',
 'Malebranche': 'AUTHOR',
 'Russell': 'AUTHOR',
 'Moore': 'AUTHOR',
 'Wittgenstein': 'AUTHOR',
 'Lewis': 'AUTHOR',
 'Quine': 'AUTHOR',
 'Popper': 'AUTHOR',
 'Kripke': 'AUTHOR',
 'Foucault': 'AUTHOR',
 'Derrida': 'AUTHOR',
 'Deleuze': 'AUTHOR',
 'Merleau-Ponty': 'AUTHOR',
 'Husserl': 'AUTHOR',
 'Heidegger': 'AUTHOR',
 'Kant': 'AUTHOR',
 'Fichte': 'AUTHOR',
 'Hegel': 'AUTHOR',
 'Marx': 'AUTHOR',
 'Lenin': 'AUTHOR',
 'Smith': 'AUTHOR',
 'Ricardo': 'AUTHOR',
 'Keynes': 'AUTHOR',
 'Epictetus': 'AUTHOR',
 'Marcus Aurelius': 'AUTHOR',
 'Nietzsche': 'SCHOOL',
 'Wollstonecraft': 'AUTHOR',
 'Beauvoir': 'AUTHOR',
 'Davis': 'AUTHOR',
 'Augustine': 'AUTHOR',
 'Anselm': 'AUTHOR',
 'Seneca': 'AUTHOR',
 'Plato - Complete Works': 'TITLE',
 'Aristotle - Complete Works': 'TITLE',
 'Second Treatise On Government': 'TITLE',
 'Essay Con

Now get an updated dropdown menu.

In [9]:
all_options = sorted([x for x in list(classifier_dict.keys())])

In [10]:
dropdown_menu = []
for source in all_options:
    dropdown_menu.append({'label': source, 'value': source})

dropdown_menu

[{'label': 'A General Theory Of Employment, Interest, And Money',
  'value': 'A General Theory Of Employment, Interest, And Money'},
 {'label': 'A Treatise Concerning The Principles Of Human Knowledge',
  'value': 'A Treatise Concerning The Principles Of Human Knowledge'},
 {'label': 'A Treatise Of Human Nature',
  'value': 'A Treatise Of Human Nature'},
 {'label': 'Analytic', 'value': 'Analytic'},
 {'label': 'Anselm', 'value': 'Anselm'},
 {'label': 'Anti-Oedipus', 'value': 'Anti-Oedipus'},
 {'label': 'Aristotle', 'value': 'Aristotle'},
 {'label': 'Aristotle - Complete Works',
  'value': 'Aristotle - Complete Works'},
 {'label': 'Augustine', 'value': 'Augustine'},
 {'label': 'Beauvoir', 'value': 'Beauvoir'},
 {'label': 'Being And Time', 'value': 'Being And Time'},
 {'label': 'Berkeley', 'value': 'Berkeley'},
 {'label': 'Beyond Good And Evil', 'value': 'Beyond Good And Evil'},
 {'label': 'Capital', 'value': 'Capital'},
 {'label': 'Capitalism', 'value': 'Capitalism'},
 {'label': 'Communi

The next cell makes pickles of each relevant slice. Not necessary since we run it all on SQL.

In [ ]:
for source in classifier_dict.keys():
        source_slice = df[df[classifier_dict[source]]==source]
        source = source.title()
        source_pkl = open(f'../search_app/slice_pickles/{source}_slice.pkl', 'wb')
        pickle.dump(source_slice, source_pkl)
        source_pkl.close()
